# Neo4j Queries
In this notebook we have performed some Neo4j queries. 
* We have computed some statistics on the Clusters, on the Addresses,   
   and on the Transactions.
* This notebook has been tested on a Bitcoin graph built with 2 BTC data files
* On the Bitcoin graph built with all the 150M transactions, some queries will   
  take long and/or you may encounter memory problems.
* Before running this notebook, you need to run Neo4j and the graph first by using   
  a SSH. 



In [ ]:
# modules
import os
import sys
import numpy as np
import pandas as pd
from py2neo import Graph

# set paths
sys.path.insert(0, "../Bitcoin_Transactions_analysis")
sys.path.insert(0, "../source")

from functions_write_save_load_data import write_table_in_latex, dump_variable
from extract_results import *

## The Graph

In [ ]:
# Connection to the Neo4j Graph (from laptop)
from py2neo import Graph

graph = Graph("bolt://localhost:7687", auth=("neo4j", "xxxxxx"))

In [ ]:
# CONNECTION TO THE GRAPH from the server
graph = Graph("bolt://134.214.108.191:7687", auth=("neo4j", "+++++++"))

In [ ]:
#  Number of nodes (labels), FAST query
q = """CALL apoc.meta.stats() YIELD labels
RETURN labels 
"""
df = graph.run(q).to_table()
df

In [ ]:
# Rels types
q = """CALL db.relationshipTypes() YIELD relationshipType as type
CALL apoc.cypher.run('MATCH ()-[:`'+type+'`]->() RETURN count(*) as count',{}) YIELD value
RETURN type, value.count"""
df = graph.run(q).to_data_frame()
df

In [ ]:
# write this result in a latex table
write_table_in_latex(df, "latex_tables.txt", "Nombres de liens de chaque catégorie")

In [ ]:
# Nodes types
q = """CALL db.labels() YIELD label
CALL apoc.cypher.run('MATCH (:`'+label+'`) RETURN count(*) as count',{}) YIELD value
RETURN label, value.count
"""
df = graph.run(q).to_data_frame()
df

In [ ]:
# write this result in a latex table
write_table_in_latex(df, "latex_tables.txt", "Nombres de Noeuds de chaque catégorie")

In [ ]:
# Total number of rels
q = """CALL apoc.meta.stats() YIELD relCount
RETURN relCount
"""

graph.run(q).to_table()

## Clusters/Actors

In [ ]:
# count the size of the cluster 1
q = """MATCH (n:Actors{cluster_size : "1"})
RETURN count(*) as count
"""
graph.run(q).to_table()

In [ ]:
# Number of clusters and their sizes
q = """MATCH (n:Actors)
RETURN n.name AS clusters, toInteger(n.cluster_size) AS size 
ORDER BY size DESC
LIMIT 10
"""
top10 = graph.run(q).to_table()
top10

In [ ]:
# smallest clusters and how many times their appears
q = """ MATCH (n:Actors)
RETURN n.cluster_size AS size, COUNT(n.cluster_size) AS counts
ORDER by counts DESC
LIMIT 10
"""
graph.run(q).to_data_frame()

In [ ]:
# higest output values for the biggest actor : actor 0
q = """MATCH p=(ac:Actors {name:'actor 0'})-[BELONG_TO_ACTOR]-(ad:Addresses)-[t:IS_IN_TRANSACTION{Tx_INS_or_OUTS:'out'}]->(tx:Transactions)
WHERE NOT t.value =  'no_value'
RETURN count(*) AS nb_outs, toInteger(t.value) AS values
ORDER BY values DESC
"""
d = graph.run(q).to_data_frame()
d.sum()

In [ ]:
# Export the result obtained using the query in a csv file
# The result is saved in Neo4j in the folder named 'import'
q = """CALL apoc.export.csv.query("MATCH p=(ac:Actors {name:'actor 0'})-[BELONG_TO_ACTOR]-(ad:Addresses)-[t:IS_IN_TRANSACTION{Tx_INS_or_OUTS:'out'}]->(tx:Transactions)
WITH DISTINCT p, t
RETURN count(*) AS nb_outs, toInteger(t.value) AS values", "actor_0.csv", {})"""

graph.run(q)

In [ ]:
# Identified Actors and their names
q = """MATCH(ac:Actors)
MATCH p=(ad:Addresses)-[r]-(ac)
WHERE NOT (ad.actor_identity = 'NA')

RETURN ac.name AS actor_ID, ac.cluster_size AS nb_Adresses, collect(DISTINCT ad.actor_identity) as Actor_Identities
LIMIT 10

"""
top10 = graph.run(q).to_table()
top10

In [ ]:
# Number of identified names for all actors
q = """MATCH (ad:Addresses)-[r]-(ac:Actors)
WHERE NOT (ad.actor_identity = 'NA')
RETURN ac.name as actor, count(*) AS nb_identities
ORDER BY nb_identities DESC
LIMIT 10
"""
df = graph.run(q).to_data_frame()
df

In [ ]:
# To convert ids to integer
q = """MATCH(ac:Actors)
SET ac.actorsId = toInteger(ac.actorsId)"""
graph.run(q)

In [ ]:
# Actors with most transactions in Input and their total values
q = """MATCH p=(ac:Actors)<-[r:BELONG_TO_ACTOR]-(ad:Addresses)-[t:IS_IN_TRANSACTION{Tx_INS_or_OUTS:'in'}]->(tx:Transactions)
WHERE NOT t.value = 'no_value'
WITH DISTINCT t, p, ac
RETURN ac.name, count(*) AS nb_txs_ins, sum(toInteger(t.value)) AS values
ORDER BY nb_txs_ins DESC
LIMIT 10
"""
graph.run(q).to_data_frame()

In [ ]:
# Actors with most transactions in Output and  their total values
q = """MATCH p=(ac:Actors)<-[r:BELONG_TO_ACTOR]-(ad:Addresses)-[t:IS_IN_TRANSACTION{Tx_INS_or_OUTS:'out'}]->(tx:Transactions)
WHERE NOT t.value = 'no_value'
WITH DISTINCT t, ac, p
RETURN ac.name, count(*) AS nb_txs_outs, sum(toInteger(t.value)) AS values
ORDER BY nb_txs_outs DESC
LIMIT 10
"""
graph.run(q).to_data_frame()

In [ ]:
# Actors appeared in most transactions (in or outs)
q = """MATCH p=(ac:Actors)<-[r:BELONG_TO_ACTOR]-(ad:Addresses)-[t:IS_IN_TRANSACTION]->(tx:Transactions)
RETURN ac.name, collect(distinct ad.actor_identity), count(*) AS nb_liens
ORDER BY nb_liens DESC
LIMIT 10
"""
graph.run(q).to_data_frame()

In [ ]:
# Top 10 biggest clusters : Sum of the of output values
q = """
MATCH p=(ac:Actors)-[BELONG_TO_ACTOR]-(ad:Addresses)-[t:IS_IN_TRANSACTION{Tx_INS_or_OUTS:'out'}]->(tx:Transactions)
WHERE ac.name >= 'actor 0' AND ac.name < 'actor 9'
RETURN ac.name as name, ac.cluster_size AS nb_addresses, count(*) AS nb_addresses_outs, sum(toInteger(t.value)) AS sum_values
ORDER BY sum_values DESC
LIMIT 10
"""
d = graph.run(q).to_data_frame()
d

## Mining 

In [ ]:
# Mining actors nodes
q = """MATCH(ad:Addresses{bitcoin_addresse:'0'})-[r]-(ac:Actors)
RETURN ac"""
graph.run(q).to_table()

In [ ]:
# nb of mining
q = """MATCH p=(ac:Actors{name:'actor 605590'})--(ad:Addresses)-[r:IS_IN_TRANSACTION]->(tx:Transactions)
//WHERE NOT r.value='no_value'
RETURN count(*) AS nb_minage
"""
graph.run(q).to_table()

In [ ]:
# nb of mining
q = """MATCH p=(ac:Actors{name:'actor 605590'})--(ad:Addresses)-[r:IS_IN_TRANSACTION]->(tx:Transactions)
WITH ad
MATCH x=(:Transactions)-[t:IS_IN_TRANSACTION{Tx_INS_or_OUTS:'in'}]-(ad)
WITH DISTINCT x
RETURN count(x)
"""
graph.run(q).to_table()

In [ ]:
q = """MATCH p=(ac:Actors{name:'actor 605590'})--(ad:Addresses)-[r:IS_IN_TRANSACTION]->(tx:Transactions)
WITH ad
MATCH x=(:Transactions)-[t:IS_IN_TRANSACTION{Tx_INS_or_OUTS:'in'}]-(ad)
RETURN x
"""
graph.run(q).to_table()

## Transactions

In [ ]:
# Top 10 : Transactions with the highest values in input
q = """MATCH p=(ad:Addresses)-[r:IS_IN_TRANSACTION{Tx_INS_or_OUTS:'in'}]->(tx:Transactions)
WHERE NOT r.value='no_value'
RETURN ad.bitcoin_addresse, toInteger(r.value) AS values, tx.name
ORDER by values DESC
LIMIT 10
"""
df = graph.run(q).to_data_frame()
df

In [ ]:
# Top 10 : Transactions with the highest values in output
q = """MATCH p=(ad:Addresses)-[r:IS_IN_TRANSACTION{Tx_INS_or_OUTS:'out'}]->(tx:Transactions)
WHERE NOT r.value='no_value'
RETURN ad.bitcoin_addresse, toInteger(r.value) AS values, tx.name
ORDER by values DESC
LIMIT 10
"""
df = graph.run(q).to_data_frame()
df

In [ ]:
# Transactions with the most diffrent actors in output
q = """MATCH p=(tx:Transactions)<-[r:IS_IN_TRANSACTION{Tx_INS_or_OUTS:'out'}]-(ad:Addresses)-->(ac:Actors)
RETURN tx.name, collect(distinct ac.name) AS lists_of_actors, count(ac) AS nb_actors
"""
graph.run(q).to_table()

In [ ]:
q = """CALL apoc.export.csv.query("MATCH p=(tx:Transactions)<-[r:IS_IN_TRANSACTION{Tx_INS_or_OUTS:'out'}]-(ad:Addresses)-->(ac:Actors)
RETURN tx.name, collect(distinct ac.name)", "txs_actors_out.csv", {})
"""
graph.run(q)

In [ ]:
# Number of transactions per timestamp
q = """MATCH(tx:Transactions)
RETURN tx.timestamp, count(tx.timestamp) AS counts
ORDER BY counts DESC
LIMIT 10"""
graph.run(q).to_table()

In [ ]:
# Number of transactions in time and the sum of total values
q = """MATCH(tx:Transactions)
RETURN tx.timestamp, count(tx.timestamp) AS nb_transactions, sum(toInteger(tx.total_value)) AS BTC_used
ORDER BY nb_transactions DESC
LIMIT 10"""
graph.run(q).to_table()

## Addresses

In [ ]:
# Bitcoin received by an address
q = """MATCH p=(ad:Addresses{bitcoin_addresse:"1eHhgW6vquBYhwMPhQ668HPjxTtpvZGPC"})-[r:IS_IN_TRANSACTION{Tx_INS_or_OUTS:'out'}]->(tx:Transactions)
RETURN sum(toInteger(r.value)) AS Satoshi_Outs"""
graph.run(q).to_data_frame()

In [ ]:
# Top 10 RICH ADDS outs. Richest adds which received bitcoins
q = """MATCH p=(ad:Addresses)-[r:IS_IN_TRANSACTION{Tx_INS_or_OUTS:'out'}]->(tx:Transactions)
RETURN ad.bitcoin_addresse AS BTC_Addresse, ad.actor_identity AS Actor_ID, sum(toInteger(r.value)) AS Satoshi_Outs
ORDER BY Satoshi_Outs DESC
LIMIT 10
"""
rich_adds = graph.run(q).to_data_frame()

In [ ]:
# Conversion from satoshi to BTC
rich_adds["BTC_Outs"] = rich_adds["Satoshi_Outs"] / 10 ** 8
rich_adds

In [ ]:
# TOP 10 RICH ADDS ins, Richest adds which sent bitcoins
q = """MATCH p=(ad:Addresses)-[r:IS_IN_TRANSACTION{Tx_INS_or_OUTS:'in'}]->(tx:Transactions)
RETURN ad.bitcoin_addresse AS BTC_Addresse, ad.actor_identity AS Actor_ID, sum(toInteger(r.value)) AS Satoshi_Ins
ORDER BY Satoshi_Ins DESC
LIMIT 10
"""
spending_adds = graph.run(q).to_data_frame()

In [ ]:

spending_adds["BTC_Ins"] = spending_adds["Satoshi_Ins"] / 10 ** 8
spending_adds

In [ ]:
# most frequent addresses in input transactions
q = """MATCH(a:Addresses)-[t:IS_IN_TRANSACTION{Tx_INS_or_OUTS:'in'}]-()
WHERE NOT t.value = 'no_value'
RETURN a.name, a.actor_identity, count(*) AS most_freq, sum(toInteger(t.value)) as sum_values
ORDER BY most_freq DESC
LIMIT 10
"""
graph.run(q).to_table()

In [ ]:
# most frequent addresses in output transactions and the sum of BTC received
q = """MATCH(a:Addresses)-[t:IS_IN_TRANSACTION{Tx_INS_or_OUTS:'out'}]-()
WHERE NOT t.value = 'no_value'
RETURN a.name, count(*) AS most_freq, sum(toInteger(t.value)) as sum_values
ORDER BY most_freq DESC
LIMIT 10
"""
graph.run(q).to_table()

In [ ]:
# BTC values for the addresse 1527626
q = """MATCH(a:Addresses{name:"addresse 1527626"})-[r:IS_IN_TRANSACTION{Tx_INS_or_OUTS:'out'}]-()
RETURN sum(toInteger(r.value)) as sum_values"""
graph.run(q).to_table()

In [ ]:
# most frequent values in input transactions and the sum of BTC sent
q = """MATCH(a:Addresses)-[r:IS_IN_TRANSACTION{Tx_INS_or_OUTS:'in'}]-()
RETURN r.value, count(*) AS most_freq
ORDER BY most_freq DESC
LIMIT 10
"""
graph.run(q).to_data_frame()